# 참고문헌
- 토닥토닥 파이썬 - 게임 만들기 (pygame) - WikiDocs
- https://wikidocs.net/book/3981
- 파이썬 Express, 천인국, 생능출판, 2020

# 1. 벽돌깨기 게임
- tkinter 이용

![벽돌깨기](images/벽돌깨기.png)

- class 정의: sprite, ball, brick, paddle, brickbreaker

## 1.1 화면 작성
- 청색 배경

In [1]:
from tkinter import * 

class BrickBreaker(Frame):
    def __init__(self, root):
        super().__init__(root)
        self.width = 640
        self.height = 480
        self.canvas = Canvas(self, bg='blue',
                                width=self.width,
                                height=self.height,)
        self.canvas.pack()
        self.pack()

window = Tk()
game = BrickBreaker(window)
window.mainloop()

## 1.2 Sprite 클래스 정의
- Sprite 클래스: 공, 벽돌, 패들(paddle)의 동일한 속성과 method를 정의
- tkinter에서 캔버스 위에 그린 도형은 식별번호로 구별
- 식별번호를 item 변수에 저장

In [2]:
class Sprite():
    def __init__(self, canvas, item):
        self.canvas = canvas	# 캔버스 객체
        self.item = item		# 캔버스 안에 있는 도형의 식별 번호
        self.speedx = 3			# x 방향 속도
        self.speedy = 3			# y 방향 속도
        self.x = 0				# 현재 x좌표  
        self.y = 0				# 현재 x좌표 

	  # 도형의 위치와 크기를 반환한다. 
    def get_coords(self):
        return self.canvas.coords(self.item)

	  # 도형의 위치를 반환한다. 
    def get_position(self):
        pos = self.canvas.coords(self.item)
        x = pos[0]
        y = pos[1]
        return x, y

	  # 객체의 상태를 변경한다. 
    def update(self):
        self.x = self.x + self.speedx
        self.y = self.y + self.speedy

	  # 객체를 움직인다. 
    def move(self):
        self.canvas.move(self.item, self.speedx, self.speedy)

	  # 객체를 캔버스에서 삭제한다. 
    def delete(self):
        self.canvas.delete(self.item)


## 1.3 Ball 클래스
- Sprite 클래스를 상속 받음
- 외부로부터 공의 위치와 크기를 받음
- 내부적으로 객체(원)을 생성하여 item에 저장
- 객체가 생성되면 부모 클래스(Sprite)의 생성자 호출
- Sprite 클래스의 update 재정의: 
    - 공의 현재 위치 이동
    - 공이 canvas 벽에 부딪치면 방향 변경

In [3]:
class Ball(Sprite):
    def __init__(self, canvas, x, y, radius):
        self.radius = radius
        item = canvas.create_oval(x-self.radius, y-self.radius,
                                  x+self.radius, y+self.radius,
                                  fill='red')
        self.x = x
        self.y = y
        super().__init__(canvas, item)		# 부모 클래스(SPrite) 생성자 호출

    def update(self):
        x, y = self.get_position()
        width = self.canvas.winfo_width()
 		 
        # 벽에 부딪히면 방향을 변경한다. 
        if x <= 0 or x >= width:
            self.speedx *= -1		# x 방향 변경
        if y <= 0:
            self.speedy *= -1		# y 방향 변경

    def collide(self, obj_list):
        x, y = self.get_position()
        
        # 공이 패들이나 벽돌에 맞으면 y방향을 반대로 한다. 
        if len(obj_list):
            self.speedy *= -1

        for obj in obj_list:
            if isinstance(obj, Brick):
                obj.handle_collision()

## 1.4 Paddle 그리기
- Sprite에서 상속 받음
- paddle 크기: width, height
- paddle은 keyboard event와 연결: 좌우 화살표 키로 작동

In [4]:
class Paddle(Sprite):
    def __init__(self, canvas, x, y):
        self.width = 100 
        self.height = 20
        item = canvas.create_rectangle(x - self.width / 2, y - self.height / 2,
                                       x + self.width / 2, y + self.height / 2,
                                       fill='white')
        super().__init__(canvas, item)  # 부모 클래스 생성자 호출
        self.x = x                      # 현재 위치 저장
        self.y = y

    # 패들을 dx, dy만큼 이동한다. 키보드 이벤트에서 호출된다. 
    def move(self, dx, dy):
        self.x = self.x + dx
        self.y = self.y + dy
        self.canvas.move(self.item, dx, dy)

## 1.5 벽돌 그리기
- Sprite에서 상속 받음
- 벽돌 크기: width, height
- handle_collision(): 공이 벽돌을 맞추는 경우 벽돌 제거, delete()

In [5]:
class Brick(Sprite):
    def __init__(self, canvas, x, y):
        self.width = 52
        self.height = 25
        item = canvas.create_rectangle(x - self.width / 2, y - self.height / 2,
                                       x + self.width / 2, y + self.height / 2,
                                       fill='yellow', tags='brick')
        super().__init__(canvas, item)

    # 벽돌과 공이 충돌하면 벽돌을 삭제한다. 
    def handle_collision(self):
            self.delete()

## 1.6 여러개의 벽돌 생성
- BrickBreaker 클래스 내에 생성
- shapes 딕셔너리: {벽돌item: 벽돌}
```
    # Brick 객체를 2차원 모양으로 생성한다. 
    for r in range(1, 4):
        for c in range(1, 10):
            brick = Brick(self.canvas, c*60, r*30)
            # Brick 객체를 shapes에 저장한다. 
            self.shapes[brick.item] = brick
```

## 1.7 paddle 움직이기
- 캔버스가 키보드 이벤트를 받으려면 focus_set() 호출
- bind() 함수를 이용하여 keyboard event와 lambda 식 연결
- lambda 식을 사용하지 않으면 즉시 함수가 호출되기 때문에 함수의 참조값을 bind()로 전달할 수 없음
```
    # 캔버스가 키보드 이벤트를 받을 수 있도록 설정한다. 
    self.canvas.focus_set()
    # 화살표키와 스페이스키에 이벤트를 붙인다.
    self.canvas.bind('<Left>',
                     lambda _: self.paddle.move(-10, 0))
    self.canvas.bind('<Right>',
                     lambda _: self.paddle.move(10, 0))
    self.canvas.bind('<space>', lambda _: self.start())
```

## 1.8 충돌 검사
- 공이 paddle에 반사되게 하려면 paddle과 충돌하였는지를 검사
- 각 객체를 감싸는 사각형이 겹치는지를 검사

![충돌검사](images/충돌검사.png)

### 공과 충돌하는 모든 객체 반환
```
    coords = self.ball.get_coords() # Ball 객체의 위치를 구한다. 
    # 겹치는 모든 도형을 찾는다. 식별 번호가 저장된다. 
    items = self.canvas.find_overlapping(*coords)

    # 겹치는 도형의 식별 번호로 객체를 찾아서 리스트에 저장한다. 
    objects = [self.shapes[x] for x in items if x in self.shapes]

    # 충돌 처리 메소드를 호출한다. 
    self.ball.collide(objects)
    self.ball.update()
    self.ball.move()
```

### 충돌 처리 메소드
- paddle과 충돌: 공이 반사 (y의 방향 반대)
- 벽돌과 충돌: 벽돌 제거(handle_collision()

```
    # Ball과 충돌이 일어난 객체들의 리스트가 매개변수로 전달된다. 
    def collide(self, obj_list):
        x, y = self.get_position()
        # 충돌이 하나라도 일어났으면 
        if len(obj_list):			# 충돌은 벽돌이나 패들
            self.speedy *= -1		# y 방향 변경

	      # 충돌이 일어난 객체가 벽돌이면 Brick의 충돌 처리 메소드를 호출한다. 
        for obj in obj_list:
            if isinstance(obj, Brick):
                obj.handle_collision()
```

## 1.9 BrickBreaker 전체 클래스
- 1.1 추가

In [6]:
class BrickBreaker(Frame):
    def __init__(self, root):
        # 벽돌깨기 frame 안의 cavas 정의
        super().__init__(root)
        self.width = 640
        self.height = 480
        self.canvas = Canvas(self, bg='blue',
                                width=self.width,
                                height=self.height,)
        self.canvas.pack()
        self.pack()
        
        # shapes에는 화면에 있는 모든 객체가 저장된다. 
        # 키는 도형 식별 번호이고 값은 객체이다. 
        self.shapes = {}

        # 패들 객체를 생성하고 shapes에 저장한다. 
        self.paddle = Paddle(self.canvas, self.width/2, 450)
        self.shapes[self.paddle.item] = self.paddle

        # Ball 객체를 생성한다. 
        self.ball = Ball(self.canvas, 310, 200, 10)

        # Brick 객체를 2차원 모양으로 생성한다. 
        for r in range(1, 4):
            for c in range(1, 10):
                brick = Brick(self.canvas, c*60, r*30)
                # Brick 객체를 shapes에 저장한다. 
                self.shapes[brick.item] = brick

        # 캔버스가 키보드 이벤트를 받을 수 있도록 설정한다. 
        self.canvas.focus_set()
        # 화살표키와 스페이스키에 이벤트를 붙인다.
        self.canvas.bind('<Left>',
                         lambda _: self.paddle.move(-10, 0))
        self.canvas.bind('<Right>',
                         lambda _: self.paddle.move(10, 0))
        self.canvas.bind('<space>', lambda _: self.start())

    def start(self):
        self.game_loop()

    # 게임 루프를 작성한다. 
    def game_loop(self):
        coords = self.ball.get_coords() # Ball 객체의 위치를 구한다. 
        # 겹치는 모든 도형을 찾는다. 식별 번호가 저장된다. 
        items = self.canvas.find_overlapping(*coords)
        
        # 겹치는 도형의 식별 번호로 객체를 찾아서 리스트에 저장한다. 
        objects = [self.shapes[x] for x in items if x in self.shapes]
        
        # 충돌 처리 메소드를 호출한다. 
        self.ball.collide(objects)
        self.ball.update()
        self.ball.move()
        
        # game_loop()를 50밀리초 후에 호출한다. 
        self.after(50, self.game_loop)

In [9]:
window = Tk()
game = BrickBreaker(window)
window.mainloop()

## 도전문제
- 점수를 화면에 표시(맞힌 블록의 갯수)
- 공이 패들에 반사될 때 남수를 추가하여 난반사
- 공의 갯수 3개
- 벽돌이 2번 맞아야 깨짐. 한번 맞을 때 색깔 변화

# 2. 두더지 게임

![두더지](images/두더지게임.PNG)

## 2.1 사용자인터페이스 화면
- 두 개의 frame
- 왼쪽 frame:두더지가 나오는 3x3의 격자
- 오른쪽 frame:두더지를 맞힌 점수, 못맞힌 점수(두 개의 label)
![두더지](images/두더지게임_interface.png)

## 2.2 게임 진행
- 3x3 격자의 버턴 image: no_mole.png
- 두더지 image: mole.png
- 버턴이 눌러지면 mole_hit() 호출 : 두더지를 맞힌 점수, 못맞힌 점수 1 증가
- 주기적(3초)으로 두더지의 위치를 random하게 변경

In [ ]:
from tkinter import *
from random import *

NUM_MOLES = 3						# 두더지 개수

window = Tk()						# 루트 윈도우 생성
window.title("두더지 게임")

moleFrame = Frame(window)			# 첫 번째 프레임 컨테이너 생성
moleFrame.grid(row=0, column=0)	# 첫 번째 프레임을 루트 윈도우에 배치

statusFrame = Frame(window)		# 두 번째 프레임 컨테이너 생성
statusFrame.grid(row=0, column=1)	# 두 번째 프레임을 루트 윈도우에 배치
hitsLabel = Label(statusFrame, text="0 Hits")
hitsLabel.pack()
missedLabel = Label(statusFrame, text="0 Misses")
missedLabel.pack()

mole_image = PhotoImage(file="images/mole.png")		# 이미지를 읽는다. 
no_mole_image = PhotoImage(file="images/no_mole.png")

numHits=0							# 획득 점수
numMissed=0						# 실패 횟수

def mole_hit(c):					# 사용자가 두더지를 잡았는지를 체크한다. 
    global numHits, numMissed, molesList, missedLabel, hitsLabel
    if molesList[c]["text"] == "mole":
        numHits += 1
        hitsLabel["text"] = str(numHits)+" Hits"
    else:
        numMissed += 1
        missedLabel["text"] = str(numMissed)+" Misses"

molesList = []						# 버튼들이 저장된다. 

def init():
    count=0
    for r in range(NUM_MOLES):		# 버튼을 만들어서 격자 형태로 배치한다. 
        for c in range(NUM_MOLES):
            button = Button(moleFrame, command=lambda c=count: mole_hit(c))
            button["image"] = no_mole_image
            button["text"] = "no mole"
            button.grid(row=r, column=c)
            molesList.append(button)
            count += 1

def update():						# 랜덤하게 버튼에 두더지를 심는다. 
    global molesList
    for i in range(NUM_MOLES*NUM_MOLES):	# 전체 버튼을 초기화한다. 
        button = molesList[i]
        button["text"] = "no mole"
        button["image"] = no_mole_image
    x = randint(0, NUM_MOLES*NUM_MOLES-1)	# 난수를 발생한다. 
    molesList[x]["image"] = mole_image		# 두더지 영상으로 바꾼다.
    molesList[x]["text"] = "mole"
    window.after(3000, update)		# 3초 지나면 다시 호출되게 한다. 

init()
update()
window.mainloop()

# 3. PyGame을 이용한 game 작성

- pygame은 SDL(Simple DirectMedia Layer) 라이브러리의 파이썬 래퍼
- SDL은 사운드, 비디오, 마우스, 키보드, 조이스틱과 같은 시스템의 기본 멀티미디어 하드웨어 구성 요소에 대한 크로스 플랫폼 액세스를 제공

![pygame](images/pygame_logo.png)

In [ ]:
# pygame 설치
!pip install pygame

## pygame 설치 확인
- DOS 창에서 "python -m pygame.examples.aliens" 실행
- C:\Anaconda3\Lib\site-packages\pygame\examples 의 다양한 예제 

## 3.1 pygame event

![pygame_event](images/pygame_event.png)

## 3.2 게임 loop
- 사용자의 입력을 처리한다. 
- 모든 게임 객체의 상태를 업데이트하고 이동시킨다.
- 디스플레이 및 오디오 출력을 업데이트한다. 
- 게임의 속도를 조절한다. 

![게임루프](images/게임loop.png)

# 4. 우주선 without class

## 4.1 게임 목표
- 참고문헌: 새내기 파이썬, 생능출판, 천인국, 2022

- 우리 우주선이 미사일로 외계 우주선을 격추하는 것이다. 
- 우리 우주선은 화면의 하단에서 왼쪽이나 오른쪽으로만 화살표 키를 이용하여서 이동한다. 
- 외계 우주선은 왼쪽에서 오른쪽으로, 위에서 아래로 자동으로 이동한다. 
- 우리 우주선은 스페이스 키를 눌러서 미사일을 발사할 수 있다. 

![우주선게임](images/우주선게임.png)

## 4.2 변수

![우주선게임](images/외계인_변수.png)

## 4.3 구현1
### 4.3.1 윈도우 생성

![우주선게임](images/우주선_구현단계1.png)

In [1]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
pygame.quit()

pygame 2.4.0 (SDL 2.26.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 4.3.2 이미지 표시하기

In [2]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))
myfont = pygame.font.SysFont('Comic Sans MS', 30)
score = 0
player=pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:    # 창 닫기 버튼 클릭 시 발생
            running = False

    display.fill((0, 0, 0))    # canva 검은색
    display.blit(player, (playerX, playerY))   # image 표시
    pygame.display.update()

pygame.quit()

### 4.3.3 우주선 움직이기
- keyboard의 좌우 화살표 키
- keyboard event 처리
- 우주선이 화면 경계를 벗어나는지 체크 (1)
- keyboard 방향키가 연속적으로 눌러진 경우 우주선의 속도 증가 (2)
- keyboard 방향키를 뗐을 경우(KEYUP) 우주선의 속도 정상 (3)

In [3]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))

player=pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:   # (2)
            if event.key == pygame.K_LEFT:    playerDx = -0.1 
            if event.key == pygame.K_RIGHT:   playerDx = 0.1
        if event.type == pygame.KEYUP:      # (3)
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                playerDx = 0

    playerX += playerDx
    if playerX <= 0 : playerX = 0           # (1)
    if playerX > 750: playerX = 750
        

    display.fill((0, 0, 0))
    display.blit(player, (playerX, playerY))
    pygame.display.update()

pygame.quit()

### 4.3.4 외계인 우주선 생성 및 동작
- 외계 우주선 화면 상단 배치 (1)
- 외계 우주선은 자율적으로 움직이면 점점 아래로 내려옴 (2)
- 외계 우주선이 화면의 좌 우 끝에서 방향을 바꿈 (3)

In [4]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))

player=pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

alien=pygame.image.load("images/alien.png")
alienX, alienY, alienDx , alienDy   = 0, 10, 0.1, 0.1     # (1)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:    playerDx = -0.1
            if event.key == pygame.K_RIGHT:   playerDx = 0.1
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                playerDx = 0

    playerX += playerDx
    if playerX <= 0 : playerX = 0 
    if playerX > 750: playerX = 750

    alienX += alienDx         
    if  alienX <= 0 or alienX > 750:     # (2)
        alienDx *= -1         # 화면의 좌 우 끝에서 방향을 바꿈 (3)
        alienY += 30
    
    display.fill((0, 0, 0))
    display.blit(player, (playerX, playerY))
    display.blit(alien, (alienX, alienY))
    pygame.display.update()

pygame.quit()

### 4.3.5 미사일 만들기
- 발사전 후 이미지 위치 변경
- 미사일 발사: SPACE key
- 발사전에는 미사일을 숨기고 움직이지 않으며, 발사후에는 미사일 나타나서 미사일 움직임.
- 미사일 상태: 'hidden', 'fire'

In [5]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))

player=pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

alien=pygame.image.load("images/alien.png")
alienX, alienY, alienDx , alienDy   = 0, 10, 0.1, 0.1

missile = pygame.image.load('images/missile.png')
missileX, missileY, missileDx , missileDy   = 0, 1000, 0, 0.1

missileState = "hidden"

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:    playerDx = -0.1
            if event.key == pygame.K_RIGHT:   playerDx = 0.1

            if event.key == pygame.K_SPACE:
                if missileState == "hidden":
                    missileState = "fire"
                    missileX, missileY = playerX, playerY   # 발사시는 우주선의 위치와 동일
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                playerDx = 0

    playerX += playerDx
    if playerX <= 0 : playerX = 0 
    if playerX > 750: playerX = 750

    alienX += alienDx
    if  alienX <= 0 or alienX > 750:
        alienDx *= -1
        alienY += 30

    if missileY <= 0:
        missileY = 1000
        missileState = "hidden"

    if missileState == "fire":
        missileY -= missileDy

    display.fill((0, 0, 0))
    display.blit(player, (playerX, playerY))

    display.blit(missile, (missileX, missileY))
    display.blit(alien, (alienX, alienY))
    pygame.display.update()

pygame.quit()

### 4.3.6 충돌 감지
- colliderect(): 객체를 둘러싸는 사각형이 겹치는지 겹치지를 검사 (1) 
- 외계인과 미사일이 충돌하면 score 1씩 증가 (2)

In [6]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))

myfont = pygame.font.SysFont('Comic Sans MS', 30)    # score용 font
score = 0    # 외계인과 미사일 충돌 횟수

player=pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

alien=pygame.image.load("images/alien.png")
alienX, alienY, alienDx , alienDy   = 0, 10, 0.1, 0.1

missile = pygame.image.load('images/missile.png')
missileX, missileY, missileDx , missileDy   = 0, 1000, 0, 0.1
missileState = "hidden"

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:    playerDx = -0.1
            if event.key == pygame.K_RIGHT:   playerDx = 0.1
            if event.key == pygame.K_SPACE:
                if missileState == "hidden":
                    missileState = "fire"
                    missileX, missileY =playerX, playerY
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                playerDx = 0

    rect1 = pygame.Rect(alien.get_rect(topleft=(alienX, alienY)))    # 외계인의 사각형
    rect2 = pygame.Rect(missile.get_rect(topleft=(missileX, missileY)))   # 미사일의 사각형
    if rect1.colliderect(rect2) and missileState != "hidden":       # (1)
        score += 1      # (2)
        alienX, alienY, alienDx , alienDy   = 0, 10, 0.1, 0.1
        
    playerX += playerDx
    if playerX <= 0 : playerX = 0 
    if playerX > 750: playerX = 750

    alienX += alienDx
    if  alienX <= 0 or alienX > 750:
        alienDx *= -1
        alienY += 30

    if missileY <= 0:
        missileY = 1000
        missileState = "hidden"

    if missileState == "fire":
        missileY -= missileDy

    display.fill((0, 0, 0))
    display.blit(player, (playerX, playerY))
    display.blit(missile, (missileX, missileY))
    display.blit(alien, (alienX, alienY))
    text = myfont.render(f'score={score}', False, (255, 255, 255))    # score 출력

    display.blit(text,(10,550))
    pygame.display.update()

pygame.quit()


### 4.3.7 외계우주선 추가 생성
- 외계우주선들은 리스트에 저장
    - alienX[ ]: 외계인 우주선의 x좌표를 저장
    - alienY[ ]: 외계인 우주선의 y좌표를 저장
    - alienDx[ ]: 외계인 우주선의 x 속도를 저장
    - alienDy[ ]: 외계인 우주선의 y 속도를 저장

In [9]:
import pygame

pygame.init()
display = pygame.display.set_mode((800, 600))
myfont = pygame.font.SysFont('Comic Sans MS', 30)
score = 0

player = pygame.image.load("images/spaceship.png")
playerX, playerY, playerDx , playerDy = 400, 550, 0, 0

alien=pygame.image.load("images/alien.png")
alienX = [ ]     # 외계우주선의 x 위치 저장
alienY = [ ]     # 외계우주선의 y 위치 저장
alienDx = [ ]    # 외계우주선의 x 축 속도 저장 
alienDy = [ ]    # 외계우주선의 y 축 속도 저장 
alienNumber = 6   

for i in range(alienNumber):
    alienX.append(20+i*60)
    alienY.append(10)
    alienDx.append(0.1)
    alienDy.append(0.0)

missile = pygame.image.load('images/missile.png')
missileX, missileY, missileDx , missileDy   = 0, 1000, 0, 0.1
missileState = "hidden"

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:              playerDx = -0.1
            if event.key == pygame.K_RIGHT:             playerDx = 0.1
            if event.key == pygame.K_SPACE:
                if missileState == "hidden":
                    missileState = "fire"
                    missileX, missileY =playerX, playerY
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                playerDx = 0

    playerX += playerDx
    if playerX <= 0 : playerX = 0 
    if playerX > 750: playerX = 750

    display.fill((0, 0, 0))
    for i in range(alienNumber):         # 모든 외계우주선 움직임
        alienX[i] += alienDx[i]
        alienY[i] += alienDy[i]
        if  alienX[i] <= 0 or alienX[i] > 750:
            alienDx[i] *= -1
            alienY[i] += 30
        rect1 = pygame.Rect(alien.get_rect(topleft=(alienX[i], alienY[i])))
        rect2 = pygame.Rect(missile.get_rect(topleft=(missileX, missileY)))
        if rect1.colliderect(rect2) and missileState != "hidden":
            score += 1
            alienX[i], alienY[i], alienDx[i] , alienDy[i] = 0, 1000, 0.1, 0.0
        display.blit(alien, (alienX[i], alienY[i]))

    if missileY <= 0:
        missileY = 1000
        missileState = "hidden"

    if missileState == "fire":
        missileY -= missileDy

    display.blit(player, (playerX, playerY))
    display.blit(missile, (missileX, missileY))
    text = myfont.render(f'score={score}', False, (255, 255, 255))

    display.blit(text,(10,550))
    pygame.display.update()

pygame.quit()

### 4.3.8 도전문제
- 미사일이 연속적으로 발사 : 리스트 사용
- 우주선이 회전하면서 하강
- 우리의 우주선이 x, y 방향으로 움직이게 (K_UP, K_DOWN) 
- 우주선, 외계우주선, 미사일을 class로 
- 효과음

# 5. 우주선 with class
- 참고문헌: 나의 첫 파이썬, 한빛미디어, 2020
- class 이용
- 게임 시나리오
    - player가 화면 아래쪽 중앙에 나타나는 우주선을 조종. 
    - player는 좌우 화살표키로 우주선 이동, space 키로 미사일 발사. 
    - 게임이 시작되면 여러 대의 외계 우주선은 좌우로 움직이면서 점점 아래로 내려온다.
    - player는 이 외계인을 격추해야 하며, 점수판에 격추 대수가 기록
    - 외계인을 모두 물리치면 이전 함대보다 더 빨리 움직이는 새로운 함대가 나타남.
    - 외계인이 우주선과 부딪히거나 화면 아래쪽에 닿으면 player는 우주선을 잃음
    - player가 세대의 우주선을 잃으면 게임 종료

## 5.1 게임 시작

- 사용자 입력에 반응하기
- settings class: 게임 요소들의 초기 설정값(크기, 배경색 등), settings.py에 정의
- 우주선 이미지 load, Ship.py에 정의

In [ ]:
import sys
import pygame

from ship import Ship            # ship.py에서 Ship 클래스 import
from settings import Settings_screen

class AlienInvasion:
    """ 게임 자원 전체의 자원과 동작을 관리하는 클래스"""

    def __init__(self):
        """게임을 초기화하고, 게임 자원을 생성"""
        pygame.init()

        self.settings = Settings_screen()   # settings.py에 정의되어 있음

        # surface: 게임 요소를 표시하는 화면 영역
        self.screen = pygame.display.set_mode(
            (self.settings.screen_width, self.settings.screen_height))
        pygame.display.set_caption("Alien Invasion")
        
        self.ship = Ship(self)   # 우주선 클래스의 instance
        
    def run_game(self):
        """게임의 main loop 시작"""
        while True:
            # 키보드와 마우스 이벤트 관찰
            for event in pygame.event.get():    # 마지막에 호출된 이후 발생한 event list
                if event.type == pygame.QUIT:
                    sys.exit()

            # loop 반복시마다 화면을 다시 그림
            self.screen.fill(self.settings.bg_color)
            self.ship.blitme()    # 우주선을 화면에 표시(Ship 클라스의 함수)
            
            # 가장 최근에 그려진 화면을 표시
            pygame.display.flip()

```
class Settings:
    """A class to store all settings for Alien Invasion."""

    def __init__(self):
        """Initialize the game's settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)
```

## 5.2 우주선 class
- image: images/ship.bmp
    - 배경색이 단색(게임배경화면과 동일)이거나 투명
![ship](images/ship.bmp)
- https://pixabay.com/ : image 무료 사용, 수정
- ship.py

```
import pygame
 
class Ship:
    """우주선을 관리하는 class"""
 
    def __init__(self, ai_game):
        """우주선을 초기화하고 시작위치 결정"""
        self.screen = ai_game.screen     # AlienInvasion의 screen
        self.settings = ai_game.settings
        # 우주선 배치를 위해 화면의 rect을 screen_rect에 할당
        self.screen_rect = ai_game.screen.get_rect()   

        # 우주선 이미지 load하고 우주선의 사각형을 가져옴.
        self.image = pygame.image.load('ship.bmp')
        self.rect = self.image.get_rect()

        # 우주선을 불러올 때마다 화면의 아래쪽 중앙에 배치
        self.rect.midbottom = self.screen_rect.midbottom

        # 우주선의 위치
        self.x = float(self.rect.x)

        # 우주선 움직임 방향
        self.moving_right = False
        self.moving_left = False

    def update(self):
        """ 우주선 움직이는 방향에 따라 우주선 위치 갱신"""
        if self.moving_right and self.rect.right < self.screen_rect.right:
            self.x += self.settings.ship_speed
        if self.moving_left and self.rect.left > 0:
            self.x -= self.settings.ship_speed

        self.rect.x = self.x

    def blitme(self):
        """현재 위치에 우주선을 그림"""
        self.screen.blit(self.image, self.rect)
```

In [ ]:
# 게임 화면의 아래쪽 중앙에 우주선 나타남
ai = AlienInvasion()
ai.run_game()

### run_game() 정비
- 기능 독립 method로 상세화
    - _check_events() : 이벤트 관리
    - _update_scree(): 화면 update
- **클래스 안에서 동작하지만 instance 외부에서는 호출되지 않음: method 이름 '_'**

    ```
    def run_game(self):
        """ 게임의 main loop을 시작 """
        while True:
            self._check_events()
            self.ship.update()
            self._update_screen()

    def _check_events(self):
        """키와 mouse 이벤트에 반응"""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _update_screen(self):
        """화면에 이미지를 갱신하고 새 화면으로 그림"""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
    ```

## 5.3 우주선 조종하기
- player가 우주선 좌우로 움직임
- key를 누르고 있으면 우주선이 화살표 방향으로 계속 움직임 
- key를 떼면 우주선 멈춤
- 우주선의 이동 범위 (화면 경계 내에서 움직임)
- Q key 누르면 종료

In [ ]:
import sys

import pygame

from settings import Settings_ship
from ship import Ship_move      # 우주선 조종 class

class AlienInvasion:
    """게임 자원 전체의 자원과 동작을 관리하는 클래스"""

    def __init__(self):
        """게임을 초기화하고, 게임 자원을 생성"""
        pygame.init()
        self.settings = Settings_ship()

        #self.screen = pygame.display.set_mode(
        #    (self.settings.screen_width, self.settings.screen_height))

        # 게임 전체화면 모드에서 실행
        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        
        pygame.display.set_caption("Alien Invasion")

        self.ship = Ship_move(self)

    def run_game(self):
        """ game의 main loop"""
        while True:
            self._check_events()
            self.ship.update()      # key event 체크 후 화면 update 전에 우주선 위치 갱신
            self._update_screen()

    def _check_events(self):
        """ Respond to keypresses and mouse events."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()

    def _check_keyup_events(self, event):
        """Respond to key releases."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _update_screen(self):
        """Update images on the screen, and flip to the new screen."""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()

        pygame.display.flip()
        
# 게임 생성, 우주선 동작 테스트
ai = AlienInvasion()
ai.run_game()

### 5.3.1 Ship_move class
- keyborad event에 따른 우주선 움직임

```
import pygame
 
class Ship_move:
    """A class to manage the ship."""
 
    def __init__(self, ai_game):
        """Initialize the ship and set its starting position."""
        self.screen = ai_game.screen
        self.settings = ai_game.settings
        self.screen_rect = ai_game.screen.get_rect()

        # Load the ship image and get its rect.
        self.image = pygame.image.load('ship.bmp')
        self.rect = self.image.get_rect()

        # Start each new ship at the bottom center of the screen.
        self.rect.midbottom = self.screen_rect.midbottom

        # 우주선 가로 위치 실수변환(ship_speed= 1.5)
        self.x = float(self.rect.x)

        # Movement flags
        self.moving_right = False
        self.moving_left = False

    def update(self):
        """움직임 falg에 따라서 우주선 위치 update"""
        # Update the ship's x value, not the rect.
        if self.moving_right and self.rect.right < self.screen_rect.right:
            self.x += self.settings.ship_speed
        if self.moving_left and self.rect.left > 0:
            self.x -= self.settings.ship_speed

        # Update rect object from self.x.
        self.rect.x = self.x

    def blitme(self):
        """Draw the ship at its current location."""
        self.screen.blit(self.image, self.rect)
```        

### 5.3.2 Settings_ship class

```
class Settings_ship:
    """A class to store all settings for Alien Invasion."""

    def __init__(self):
        """Initialize the game's settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)

        # Ship settings
        self.ship_speed = 1.5
```        

## 5.4 탄환 발사하기
- space key를 누르면 탄환발사
- 탄환은 상단을 향해 직선으로 날아가며, 화면의 상단에 도착하면 사라짐
- 한 화면에 존재하는 탄환 수 제한(Settings_bullet class)

### 5.4.1 Settings_bullet class

```
class Settings_bullet:
    """A class to store all settings for Alien Invasion."""

    def __init__(self):
        """Initialize the game's settings."""
        # Screen settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)

        # Ship settings
        self.ship_speed = 1.2

        # Bullet settings
        self.bullet_speed = 1.0
        self.bullet_width = 3
        self.bullet_height = 15
        self.bullet_color = (60, 60, 60)
        self.bullets_allowed = 3    # 한 화면에 존재하는 탄환 수
```

### 5.4.2 Bullet class
- sprite class 사용: 게임에서 관련된 요소들을 하나로 묶어 한번에 관리
- 탄환은 이미지가 아닌 사각형을 직접 만듦.
- 탄환의 발사 위치는 우주선의 위치
- 탄환의 midtop 속성을 우주선의 midtop 속성과 같게

```
import pygame
from pygame.sprite import Sprite
 
class Bullet(Sprite):
    """A class to manage bullets fired from the ship"""

    def __init__(self, ai_game):
        """Create a bullet object at the ship's current position."""
        super().__init__()
        self.screen = ai_game.screen
        self.settings = ai_game.settings
        self.color = self.settings.bullet_color

        # Create a bullet rect at (0, 0) and then set correct position.
        self.rect = pygame.Rect(0, 0, self.settings.bullet_width,
            self.settings.bullet_height)
        self.rect.midtop = ai_game.ship.rect.midtop
        
        # Store the bullet's position as a decimal value.
        self.y = float(self.rect.y)

    def update(self):
        """Move the bullet up the screen."""
        # Update the decimal position of the bullet.
        self.y -= self.settings.bullet_speed
        # Update the rect position.
        self.rect.y = self.y

    def draw_bullet(self):
        """Draw the bullet to the screen."""
        pygame.draw.rect(self.screen, self.color, self.rect)
```

In [ ]:
import sys

import pygame

from settings import Settings_bullet
from ship import Ship_move
from bullet import Bullet

class AlienInvasion:
    """Overall class to manage game assets and behavior."""

    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init()
        self.settings = Settings_bullet()

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion")

        self.ship = Ship_move(self)
        # 발사된 탄환을 모두 저장
        self.bullets = pygame.sprite.Group()

    def run_game(self):
        """Start the main loop for the game."""
        while True:
            self._check_events()
            self.ship.update()
            self._update_bullets()  # 그룹이 자동으로 sprite 전체 update
            self._update_screen()

    def _check_events(self):
        """Respond to keypresses and mouse events."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            sys.exit()
        elif event.key == pygame.K_SPACE:   # 탄환 발사
            self._fire_bullet()

    def _check_keyup_events(self, event):
        """Respond to key releases."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """ 새 탄환 생성 & bullets group에 추가 """
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

    def _update_bullets(self):
        """ 탄환 위치 갱신 & 사라진 탄환 제거 """
        # Update bullet positions.
        self.bullets.update()

        # Get rid of bullets that have disappeared.
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                 self.bullets.remove(bullet)

    def _update_screen(self):
        """Update images on the screen, and flip to the new screen."""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        pygame.display.flip()

ai = AlienInvasion()
ai.run_game()

## 5.5 외계인
### 5.5.1 Alien Class
![외계인](images/alien.png)
- https://nostarch.com/pythoncrashcourse2e/
- 외계인의 위치: 왼쪽 상단 모서리

```
import pygame
from pygame.sprite import Sprite
 
class Alien(Sprite):
    """A class to represent a single alien in the fleet."""

    def __init__(self, ai_game):
        """Initialize the alien and set its starting position."""
        super().__init__()
        self.screen = ai_game.screen

        # Load the alien image and set its rect attribute.
        self.image = pygame.image.load('images/alien.bmp')
        self.rect = self.image.get_rect()

        # Start each new alien near the top left of the screen.
        self.rect.x = self.rect.width
        self.rect.y = self.rect.height

        # Store the alien's exact horizontal position.
        self.x = float(self.rect.x)
```

In [ ]:
import sys

import pygame

from settings import Settings_bullet
from ship import Ship_move
from bullet import Bullet
from alien import Alien

class AlienInvasion:
    """Overall class to manage game assets and behavior."""

    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init()
        self.settings = Settings_bullet()

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion")

        self.ship = Ship_move(self)
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()

        self._create_fleet()

    def run_game(self):
        """Start the main loop for the game."""
        while True:
            self._check_events()
            self.ship.update()
            self._update_bullets()
            self._update_screen()

    def _check_events(self):
        """Respond to keypresses and mouse events."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()

    def _check_keyup_events(self, event):
        """Respond to key releases."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """Create a new bullet and add it to the bullets group."""
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

    def _update_bullets(self):
        """Update position of bullets and get rid of old bullets."""
        # Update bullet positions.
        self.bullets.update()

        # Get rid of bullets that have disappeared.
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                 self.bullets.remove(bullet)

    def _create_fleet(self):
        """ 외계인 함대 생성 """
        # Make an alien.
        alien = Alien(self)
        self.aliens.add(alien)

    def _update_screen(self):
        """Update images on the screen, and flip to the new screen."""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)     # 외계인 출력

        pygame.display.flip()

ai = AlienInvasion()
ai.run_game()

### 5.5.2 외계인 함대 만들기
- *def _create_fleet(self)* 에 정의
- 한줄의 좌 우에 외계인 하나의 너비만큼 margin: available_space_x
- 외계인과 외계인 사이의 공간: 외계인 하나의 너비
- 한줄의 외계인 숫자: number_aliens_x


- 여러 줄의 외계인 생성
    - 줄간 간격: 외계인 높이
    - 외계인 생성 가용한 공간: available_space_y
    - 외계인 줄 수 : number_rows

In [1]:
import sys

import pygame

from settings import Settings_bullet
from ship import Ship_move
from bullet import Bullet
from alien import Alien

class AlienInvasion:
    """Overall class to manage game assets and behavior."""

    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init()
        self.settings = Settings_bullet()

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion")

        self.ship = Ship_move(self)
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()

        self._create_fleet()

    def run_game(self):
        """Start the main loop for the game."""
        while True:
            self._check_events()
            self.ship.update()
            self._update_bullets()
            self._update_screen()

    def _check_events(self):
        """Respond to keypresses and mouse events."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()

    def _check_keyup_events(self, event):
        """Respond to key releases."""
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """Create a new bullet and add it to the bullets group."""
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

    def _update_bullets(self):
        """Update position of bullets and get rid of old bullets."""
        # Update bullet positions.
        self.bullets.update()

        # Get rid of bullets that have disappeared.
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                 self.bullets.remove(bullet)

    def _create_fleet(self):
        """Create the fleet of aliens."""
        # 외계인 하나를 만들고 한 줄에 몇개의 외계인이 들어갈 지 계산 
        # 외계인 사이의 공간은 외계인 하나의 너비
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width)
        
        # Determine the number of rows of aliens that fit on the screen.
        ship_height = self.ship.rect.height
        available_space_y = (self.settings.screen_height -
                                (3 * alien_height) - ship_height)
        number_rows = available_space_y // (2 * alien_height)
        
        # Create the full fleet of aliens.
        for row_number in range(number_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)

    def _create_alien(self, alien_number, row_number):
        """Create an alien and place it in the row."""
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        alien.x = alien_width + 2 * alien_width * alien_number
        alien.rect.x = alien.x
        alien.rect.y = alien.rect.height + 2 * alien.rect.height * row_number
        self.aliens.add(alien)

    def _update_screen(self):
        """Update images on the screen, and flip to the new screen."""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)     # 외계인 출력

        pygame.display.flip()

ai = AlienInvasion()
ai.run_game()

pygame 2.0.0 (SDL 2.0.12, python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

### 5.5.3 외계인 함대 움직이기
- 왼쪽에서 오른쪽으로 이동
- 스크린의 끝에서는 조금 내린 다음 반대편으로 이동

# 지뢰 찾기 게임

# 동굴 게임

- 참고: https://wikidocs.net/63761 토닥토닥 파이썬 - 게임 만들기 (pygame)